In [ ]:
# load dependencies

## first time series test
# append ../../ to make link to main repo work
import sys
sys.path.append('../../')
sys.path.append('/Applications/anaconda3/envs/PhD_clean/lib/python3.11/site-packages/')


# import external packages
import numpy as np
import pandas as pd
import numba
from numba import vectorize
import glob # for file search
import copy
import os # operating system stuff
import re # regex


import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio.plot import plotting_extent
# import rioxarray
from shapely.geometry import Polygon
from shapely.geometry.point import Point
import pyproj
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 
import matplotlib.image as mplimg

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
from matplotlib import rc_file_defaults
rc_file_defaults()
# sns.set(style=None, color_codes=True)

from shapely.geometry import Polygon
from shapely.geometry.point import Point
import datetime

import configparser

from cmcrameri import cm # for scientific colourmaps

from scipy import spatial
# from scikit_spatial import Line as sksline
# import scipy.spatial.transform 

from skspatial.objects import Line as sksline
import scipy.spatial.transform 

import json
from shapely.geometry import Point, LineString, mapping
from functools import partial
from shapely.ops import transform

import sys
sys.executable


###########################
# import main local package
import SPOTSAR_main as sm



In [ ]:
### steps:

# 1. load rasters
# 1.1 get dates from file
# 2. load lavaflows and craters
# 3. plot results in grid

In [ ]:
# 1. load rasters

Data_dir  ="/Users/tz20896/Documents/PhD/projects/Merapi2021/S1/daisychain/rasters/"

file_list = glob.glob(data_dir)


### get EDM positions
pos_bad = np.array([110.410626, -7.526175])
reflectors = np.array(
    [[110.439757, -7.537567], [110.440165, -7.537451], [110.439260, -7.537533]]
)

### lava flows and crater
L1888_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1888_v2.shp")
L1948_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1948.shp")
L1956_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1956.shp")
L1992_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1992.shp")
L1997_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1997.shp")
L1998_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1998.shp")
L1986_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1986.shp")
L1984_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1984.shp")
L1931_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1931.shp")

OLD_CRATER_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/old_crater_rims.shp")
CRATER_FILE = "/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/Merapi_crater.shp"

# Extract latitude and longitude into separate columns
coords_L1888   = sm.misc_func.read_line_shp(L1888_FILE )
coords_L1931   = sm.misc_func.read_line_shp(L1931_FILE )
coords_L1948   = sm.misc_func.read_line_shp(L1948_FILE )
coords_L1956   = sm.misc_func.read_line_shp(L1956_FILE )
coords_L1984   = sm.misc_func.read_line_shp(L1956_FILE )
coords_L1986   = sm.misc_func.read_line_shp(L1956_FILE )
coords_L1992   = sm.misc_func.read_line_shp(L1992_FILE )
coords_L1997   = sm.misc_func.read_line_shp(L1997_FILE )
coords_L1998   = sm.misc_func.read_line_shp(L1998_FILE )
coords_CRATER  = sm.misc_func.read_line_shp(CRATER_FILE)
coords_OLD_CRATER  = sm.misc_func.read_line_shp(OLD_CRATER_FILE)


# separate multi-line
coords_L1956_1 = coords_L1956[0]
coords_L1956_2 = coords_L1956[1]

coords_pas1_CRATER   = coords_OLD_CRATER[0]
coords_pas2_CRATER   = coords_OLD_CRATER[1]
coords_1822_CRATER   = coords_OLD_CRATER[2]
coords_1872_CRATER   = coords_OLD_CRATER[3]
coords_1904_1_CRATER = coords_OLD_CRATER[4]
coords_1904_2_CRATER = coords_OLD_CRATER[5]
coords_1932_CRATER   = coords_OLD_CRATER[6]
coords_1961_1_CRATER = coords_OLD_CRATER[7]
coords_1961_2_CRATER = coords_OLD_CRATER[8]



## DEM data
# DEM
Merapi_DEM_file = "/Users/tz20896/Documents/PhD/projects/Merapi2021/geo/TDX_Merapi_WGS84_5m.tif"
with rio.open(Merapi_DEM_file) as img:
    DEM = img.read(1,masked=True) # rasterio bands are indexed from 1
    # extract DEM extent
    DEM_extent=[img.bounds.left,img.bounds.right,img.bounds.bottom,img.bounds.top]
    print('Band1 has shape', DEM.shape)
    height = DEM.shape[0]
    width  = DEM.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rio.transform.xy(img.transform, rows, cols)
    lons_DEM = np.array(xs)
    lats_DEM = np.array(ys)


# DEM
Merapi_HS_file = "/Users/tz20896/Documents/PhD/projects/Merapi2021/geo/TDX_Merapi_WGS84_HS.tif"
with rio.open(Merapi_HS_file) as img:
    SHADING = img.read(1,masked=True) # rasterio bands are indexed from 1






## defien query points fro time series plot
Q_points = [(110.44595,-7.53895),
            (110.44364,-7.53659),
            (110.440165,-7.537451),
            (110.44000,-7.53831),
            (110.44322,-7.54022),
            (110.44280,-7.54211),
            (110.43365,-7.54146),
            (110.44493,-7.54096),
            (110.44611,-7.54102),
            (110.44894,-7.53674),
            ]
